In [ ]:
# environment setting 
# https://huggingface.co/course/chapter0/1?fw=pt

!pip install transformers sentencepiece datasets nltk evaluate torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.4 MB/s 
     |████████████████████████████████| 1.3 MB 33.7 MB/s 
     |████████████████████████████████| 441 kB 71.2 MB/s 
     |████████████████████████████████| 72 kB 1.6 MB/s 
     |████████████████████████████████| 7.6 MB 37.1 MB/s 
     |████████████████████████████████| 163 kB 14.6 MB/s 
     |████████████████████████████████| 115 kB 53.0 MB/s 
     |████████████████████████████████| 212 kB 55.4 MB/s 
     |████████████████████████████████| 127 kB 48.2 MB/s 
     |████████████████████████████████| 115 kB 54.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import transformers
import torch
import numpy as np
import nltk
nltk.download('punkt')

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from evaluate import load

# preprocssing. this encodes command and actions 
def preprocess_function(examples):

    model_inputs = tokenizer(examples["commands"], max_length=tokenizer.model_max_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["actions"], max_length=tokenizer.model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# training function, returns the model (which is trainer)
def model_train():
  # define args for finetuning
  batch_size = 8
  args = Seq2SeqTrainingArguments(
      f"{model_name}-scan-finetuned-yoon-1027",
      evaluation_strategy = "epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      weight_decay=0.01,
      save_total_limit=3,
      num_train_epochs=20, # or 20
      predict_with_generate=True,
      # fp16=True,
      # push_to_hub=True,
  )

  # collator fixed of max length 
  data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

  trainer = Seq2SeqTrainer(
      model,
      args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["test"],
      data_collator=data_collator,
      tokenizer=tokenizer,
  )

  trainer.train()

  return trainer    

# decoding function, this converts list of intergers into sentence 
def decode_data(prediction, label):
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them.
  labels = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # We convert back into the sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

  print('##################')
  print('decoded prediction: ', decoded_preds[0])
  print('decoded label: ', decoded_labels[0])

  return decoded_preds, decoded_labels

######### truncate function: THIS FUNCTION SHOULD BE REMOVED 
# this is actually wrong, as we are truncating the actual result to match the length
# We don't want to use this function, but this is needed as per now 
def truncate_data(decoded_preds, decoded_labels):

  test_preds = []
  test_labels = []

  for i in range(len(decoded_labels)):
      test_preds.append(decoded_preds[i][:len(decoded_labels[i])
      test_labels.append(decoded_labels[i])

  return test_preds, test_labels

# evaluate function: this calculates exact match score 
def evaluate_exact_match(decoded_preds, decoded_labels):
  exact_match_metric = load("exact_match")

  match = exact_match_metric.compute(predictions=decoded_preds, references=decoded_labels)['exact_match']
  print('Exact match: ', match)

  return match


if __name__ == "__main__":
  # dataset 
  raw_datasets = load_dataset("scan", "simple")
  
  ### smoke test
  # raw_datasets['train'] = raw_datasets["train"].shuffle(seed=42).select(range(500))
  # raw_datasets['test'] = raw_datasets["test"].shuffle(seed=42).select(range(500))
  
  # model name, this can be changed 
  model_checkpoint = "t5-small" 
  model_name = model_checkpoint.split("/")[-1]

  # define tokenizer, model 
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

  # preprocess dataset
  tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)  

  # train and predict 
  trainer = model_train()

  # creating max length among test labels
  max_length = 0
  for i in tokenized_datasets['test']['labels']:
    if len(i) > max_length: 
      max_length = len(i)

  # predict: this needs to be fixed, but we are not sure how to work on it 
  # by default, it truncates the prediction into max_length = 20, but we are not sure why this is happening
  predictions, label_ids, _ = trainer.predict(tokenized_datasets["test"], max_length=max_length) 

  print('prediction: ', predictions[0])
  print('label: ', label_ids[0])

  # decoding results 
  decoded_preds, decoded_labels = decode_data(predictions, label_ids)  

  #### IMPORTANT #####################################
  #### truncate needed as per now, but want to figure out this part
  test_preds, test_labels = truncate_data(decoded_preds, decoded_labels)
  print('truncated_prediction: ', test_preds[0])
  print('truncated_label: ', test_labels[0])

  # evaluate exact match result 
  match = evaluate_exact_match(test_preds, test_labels)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/2 [00:00<?, ?it/s]

...

Saving model checkpoint to t5-small-scan-finetuned-yoon-1027/checkpoint-500
Configuration saved in t5-small-scan-finetuned-yoon-1027/checkpoint-500/config.json
Model weights saved in t5-small-scan-finetuned-yoon-1027/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-scan-finetuned-yoon-1027/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-scan-finetuned-yoon-1027/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-small-scan-finetuned-yoon-1027/checkpoint-500/spiece.model
Saving model checkpoint to t5-small-scan-finetuned-yoon-1027/checkpoint-1000
Configuration saved in t5-small-scan-finetuned-yoon-1027/checkpoint-1000/config.json
Model weights saved in t5-small-scan-finetuned-yoon-1027/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-scan-finetuned-yoon-1027/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-small-scan-finetuned-yoon-1027/checkpoint-1000/special_tokens_map.json
Copy v

Epoch,Training Loss,Validation Loss
1,0.108600,0.052165
2,0.058000,0.030111
3,0.038900,0.031016
4,0.030400,0.012465
5,0.025600,0.012969
6,0.021000,0.009573
7,0.017700,0.011620
8,0.015900,0.008016
9,0.014500,0.013507
10,0.013400,0.008969


Saving model checkpoint to t5-small-scan-finetuned-yoon-1027/checkpoint-24500
Configuration saved in t5-small-scan-finetuned-yoon-1027/checkpoint-24500/config.json
Model weights saved in t5-small-scan-finetuned-yoon-1027/checkpoint-24500/pytorch_model.bin
tokenizer config file saved in t5-small-scan-finetuned-yoon-1027/checkpoint-24500/tokenizer_config.json
Special tokens file saved in t5-small-scan-finetuned-yoon-1027/checkpoint-24500/special_tokens_map.json
Copy vocab file to t5-small-scan-finetuned-yoon-1027/checkpoint-24500/spiece.model
Deleting older checkpoint [t5-small-scan-finetuned-yoon-1027/checkpoint-23000] due to args.save_total_limit
Saving model checkpoint to t5-small-scan-finetuned-yoon-1027/checkpoint-25000
Configuration saved in t5-small-scan-finetuned-yoon-1027/checkpoint-25000/config.json
Model weights saved in t5-small-scan-finetuned-yoon-1027/checkpoint-25000/pytorch_model.bin
tokenizer config file saved in t5-small-scan-finetuned-yoon-1027/checkpoint-25000/tokeniz

prediction:  [    0    27   834 21164   567   834 27262    27   834 21164   567   834
 27262    27   834 21164   567   834 27262    27   834 21164   567   834
 27262    27   834 21164   567   834 27262    27   834 21164   567   834
 27262    27   834 21164   567   834  3765  6245    27   834 21164   567
   834  3765  6245     1     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0  

Exact match:  0.966284074605452


In [ ]:
print('truncated_prediction: ', test_preds[0])
print('truncated_label: ', test_labels[0])

truncated_prediction:  I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT
truncated_label:  I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT


In [ ]:
len(test_preds[0]), len(test_labels[0])

(101, 101)

In [ ]:
import gc
gc.collect()

1985